# 토큰화

In [ ]:
en_text = "A Dog Run back corner near spare bedrooms"

In [ ]:
# spaCy 사용

import spacy
spacy_en = spacy.load('en_core_web_sm')

def tokenize(en_text):
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

print(tokenize(en_text))

In [ ]:
# NLTK 사용

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

In [ ]:
# 띄어쓰기로 분리

print(en_text.split())

In [ ]:
# 한국어 띄어쓰기 토큰화
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사왔어"
print(kor_text.split())

# 조사가 붙어 있어서 제거 안하면 전부 다른 단어로 인식

In [ ]:
# 형태소 토큰화

!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

In [ ]:
# 문자 토큰화

print(list(en_text))

In [ ]:
# 단어 집합(Vocabulary) 생성

import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
data[:10]

print('전체 샘플의 수 : {}'.format(len(data)))

sample_data = data[:100]

sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)

# 한글과 공백을 제외하고 모두 제거
sample_data[:10]

# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenizer = Mecab()
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)
    
print(tokenized[:10])

vocab = FreqDist(np.hstack(tokenized))
print('단어 집합의 크기 : {}'.format(len(vocab)))

vocab['재밌']

vocab_size = 500
# 상위 vocab_size개의 단어만 보존
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기 : {}'.format(len(vocab)))

In [ ]:
# 각 단어에 고유한 정수 부여

word_to_index = {word[0]:index+1 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index

encoded = []
for line in tokenized: # 입력 데이터에서 1줄씩 문장을 읽음
    temp = []
    for w in line: # 문장에서 단어를 하나씩 읽음
        try:
            temp.append(word_to_index[w]) # 글자에 해당되는 정수로 변환
        except KeyError: # 단어 집합에 없는 단어는 unk로 대체
            temp.append(word_to_index['unk']) # unk의 인덱스로 변환
    encoded.append(temp)
    
print(encoded[:10])

In [ ]:
# 길이가 다른 문장들을 모두 동일한 길이로 바꿔주는 패딩

max_len = max(len(l) for l in encoded)
print('리뷰의 최대 길이 : %d' % max_len)
print('리뷰의 최소 길이 : %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))
plt.hist([len(s) for s in encoded], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

for line in encoded:
    if len(line) < max_len: # 현재 샘플이 정해준 길이보다 짧으면
        line += [word_to_index['pad']] * (max_len - len(line)) # 나머지는 전부 pad로 채움
        
print('리뷰의 최대 길이 : %d' % max(len(l) for l in encoded))
print('리뷰의 최소 길이 : %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))

print(encoded[:3])